In [ ]:
from lsst.daf.butler import Butler
import lsst.afw.display as afwDisplay
import lsst.afw.image as afwImage
import matplotlib.pyplot as plt
from lsst.ip.isr.isrTask import IsrTask
import numpy
from matplotlib.patches import Rectangle
import pylab
from astropy.stats import mad_std


In [ ]:
def display(exposure,title=None,frame=0):
    afw_display = afwDisplay.Display(frame=frame, backend='matplotlib')
    afw_display.scale('asinh', 'zscale')
    afw_display.setMaskTransparency(20)
#    afw_display.mtv(exposure.getMaskedImage())
    afw_display.mtv(exposure.getImage())
    
    maskPlaneDict = exposure.getMask().getMaskPlaneDict()
    maskKeyColorList = []
    for k in maskPlaneDict.keys():
        if afw_display.getMaskPlaneColor(k) is not None:
            maskKeyColorList.append(k + ":" + afw_display.getMaskPlaneColor(k))
    print(maskKeyColorList)
    plt.title(title)



In [ ]:
butler = Butler('/repo/LATISS', collections=["LATISS/raw/all", "LATISS/calib", "LATISS/runs/quickLook",'LATISS/calib/daily'])
dataId = dict(day_obs=20240223, detector=0, seq_num=577)

In [ ]:
from lsst.summit.utils.bestEffort import BestEffortIsr
bestEffort = BestEffortIsr()

be=bestEffort.getExposure(dataId)

In [ ]:
%matplotlib inline
display(be)

In [ ]:
def display(exposure,title=None):
    afwDisplay.setDefaultBackend('matplotlib') 
    fig = plt.figure(figsize=(10,10))
    afw_display = afwDisplay.Display(1)
    afw_display.scale('linear', 'zscale')
    afw_display.mtv(exposure.getImage())
    plt.title(title)

In [ ]:
from lsst.summit.utils import quickSmooth
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
def plotExp(exposure):
    # Borrowed from summit utils
    data = quickSmooth(exposure.image.array, 1)
    vmin = np.nanpercentile(data, 1)
    vmax = np.nanpercentile(data, 99)
    print(vmin, vmax)
    figure = plt.figure(figsize=(12,12))
    figure.clear()
    ax1 = figure.add_subplot(111)
    im1 = ax1.imshow(data, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
    ax1.tick_params(which="major", direction="in", top=True, right=True, labelsize=8)
    divider = make_axes_locatable(ax1)
    cax = divider.append_axes("right", size="5%", pad=0.05)
    plt.colorbar(im1, cax=cax)
    plt.tight_layout()
    return figure, vmin, vmax


In [ ]:
plotExp(be)
plt.savefig("Test.png")